<a href="https://colab.research.google.com/github/Swyam17/LlmTokenizers/blob/main/LlmTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**S1**: CREATING TOKENS

In [4]:
with open("/content/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20781
THE VERDICT
June 1908

I had always thought Jack Gisburn rather a cheap genius--though a

good fell


In [5]:
import re
text = "Hello, world. This , is a test."
result = re.split(r'(\s)',text) # this is used to split the text and even the blank spaces
print(result)

['Hello,', ' ', 'world.', ' ', 'This', ' ', ',', ' ', 'is', ' ', 'a', ' ', 'test.']


In [6]:
import re
text = "Hello, world. This , is a test."
result = re.split(r'([,.]|\s)',text) # couma is seperate token
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ' ', '', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [7]:
result = [item for item in result if item.strip()] #removes blank space
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


REMOVING WHITESPACES OR NOT


When developing a simple tokenizer, whether we should encode whitespaces as
separate characters or just remove them depends on our application and its
requirements. Removing whitespaces reduces the memory and computing
requirements. However, keeping whitespaces can be useful if we train models that
are sensitive to the exact structure of the text (for example, Python code, which is
sensitive to indentation and spacing). Here, we remove whitespaces for simplicity
and brevity of the tokenized outputs. Later, we will switch to a tokenization scheme
that includes whitespaces.

TOKENIZATION

The tokenization scheme we devised above works well on the simple sample text. Let's
modify it a bit further so that it can also handle other types of punctuation, such as
question marks, quotation marks, and the double-dashes we have seen earlier in the first
100 characters of Edith Wharton's short story, along with additional special characters:

In [8]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item for item in result if item.strip()] #removes blank space
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['THE', 'VERDICT', 'June', '1908', 'I', 'had', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to']


In [12]:
print(len(preprocessed))

4667


**S2**: CREATING TOKEN IDS

In [13]:
all_words = sorted(set(preprocessed)) #sort the words
vocab_size = len(all_words) #length of words
print(vocab_size)

1148


In [14]:
vocab ={token: integer for integer, token in enumerate(all_words)}

In [15]:
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
('1908', 8)
(':', 9)
(';', 10)
('?', 11)
('A', 12)
('AM', 13)
('Ah', 14)
('Among', 15)
('And', 16)
('Are', 17)
('Arrt', 18)
('As', 19)
('At', 20)
('Be', 21)
('Begin', 22)
('Burlington', 23)
('But', 24)
('By', 25)
('Carlo', 26)
('Chicago', 27)
('Claude', 28)
('Come', 29)
('Croft', 30)
('Destroyed', 31)
('Devonshire', 32)
('Don', 33)
('Dubarry', 34)
('Emperors', 35)
('End', 36)
('FELT', 37)
('Florence', 38)
('For', 39)
('Gallery', 40)
('Gideon', 41)
('Gisburn', 42)
('Gisburns', 43)
('Grafton', 44)
('Greek', 45)
('Grindle', 46)
('Grindles', 47)
('HAD', 48)
('HAS', 49)
('HAVE', 50)
